In [1]:
import numpy as np
import pandas as pd

df = pd.read_excel("特征归一化.xlsx",index_col=0)
df

,Word,1 try,2 tries,3 tries,4 tries,5 tries,6 tries,7 or more tries (X),e,t,a,o,i,th,he,in,er,an,Frequency,Char_repitions
0,slump,1,3,23,39,24,9,1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.000661,0.0
1,crank,1,5,23,31,24,14,2,0.0,0.0,0.5,0.0,0.0,0,0,0,0,1,0.000765,0.0
2,gorge,1,3,13,27,30,22,4,0.5,0.0,0.0,0.5,0.0,0,0,0,0,0,0.001304,0.5
3,query,1,4,16,30,30,17,2,0.5,0.0,0.0,0.0,0.0,0,0,0,1,0,0.005003,0.0
4,drink,1,9,35,34,16,5,1,0.0,0.0,0.0,0.0,0.5,0,0,1,0,0,0.027824,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,condo,0,2,17,35,29,14,3,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0.000951,0.5
355,impel,0,3,21,40,25,9,1,0.5,0.0,0.0,0.0,0.5,0,0,0,0,0,0.000121,0.0
356,havoc,0,2,16,38,30,12,2,0.0,0.0,0.5,0.5,0.0,0,0,0,0,0,0.001178,0.0
357,molar,0,4,21,38,26,9,1,0.0,0.0,0.5,0.5,0.0,0,0,0,0,0,0.001504,0.0


In [31]:
zhibiao = df.drop(df.columns[4:18],axis=1).drop(df.columns[0],axis=1)

In [32]:
P = zhibiao/zhibiao.sum()
e = np.nan_to_num(-P*np.log(P)).sum(axis=0)/np.log(len(P)) # 这里会出现空值，有一个概率为0，0*ln0由极限可知为0，但计算机求出来为0*-inf=nan
d = 1 - e
W = d/d.sum() # weight权重
pd.DataFrame(W,index=zhibiao.columns,columns=["熵权法确定的权重"])

f:\Anaconda\envs\tensorflow\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,熵权法确定的权重
1 try,0.230799
2 tries,0.045855
3 tries,0.013156
Frequency,0.434949
Char_repitions,0.275241


In [34]:
# 此时的df是已经正向化后的数据
df_rank=pd.DataFrame()
for i, X in enumerate(zhibiao.columns):
    df_rank[f'X{str(i + 1)}:{X}'] = zhibiao.iloc[:, i]
    df_rank[f'R{str(i + 1)}:{X}'] = zhibiao.iloc[:, i].rank(method="dense")
df_rank

,X1:1 try,R1:1 try,X2:2 tries,R2:2 tries,X3:3 tries,R3:3 tries,X4:Frequency,R4:Frequency,X5:Char_repitions,R5:Char_repitions
0,1,2.0,3,4.0,23,19.0,0.000661,98.0,0.0,1.0
1,1,2.0,5,6.0,23,19.0,0.000765,115.0,0.0,1.0
2,1,2.0,3,4.0,13,9.0,0.001304,153.0,0.5,2.0
3,1,2.0,4,5.0,16,12.0,0.005003,247.0,0.0,1.0
4,1,2.0,9,10.0,35,31.0,0.027824,318.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
354,0,1.0,2,3.0,17,13.0,0.000951,127.0,0.5,2.0
355,0,1.0,3,4.0,21,17.0,0.000121,17.0,0.0,1.0
356,0,1.0,2,3.0,16,12.0,0.001178,146.0,0.0,1.0
357,0,1.0,4,5.0,21,17.0,0.001504,169.0,0.0,1.0


In [35]:
# 利用熵权法得到的权重计算WRSR
m, n = zhibiao.shape
df_rank["WRSR"] = (df_rank.iloc[:,1::2]*W).sum(axis=1)/n # 如果只使用RSR,把这里的W去掉即可
df_rank["Rank"] = df_rank["WRSR"].rank(ascending=False,method="dense")
df_rank

,X1:1 try,R1:1 try,X2:2 tries,R2:2 tries,X3:3 tries,R3:3 tries,X4:Frequency,R4:Frequency,X5:Char_repitions,R5:Char_repitions,WRSR,Rank
0,1,2.0,3,4.0,23,19.0,0.000661,98.0,0.0,1.0,8.759048,262.0
1,1,2.0,5,6.0,23,19.0,0.000765,115.0,0.0,1.0,10.256217,245.0
2,1,2.0,3,4.0,13,9.0,0.001304,153.0,0.5,2.0,13.572226,207.0
3,1,2.0,4,5.0,16,12.0,0.005003,247.0,0.0,1.0,21.711287,113.0
4,1,2.0,9,10.0,35,31.0,0.027824,318.0,0.0,1.0,27.983412,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...
354,0,1.0,2,3.0,17,13.0,0.000951,127.0,0.5,2.0,11.265684,233.0
355,0,1.0,3,4.0,21,17.0,0.000121,17.0,0.0,1.0,1.661449,343.0
356,0,1.0,2,3.0,16,12.0,0.001178,146.0,0.0,1.0,12.860811,214.0
357,0,1.0,4,5.0,21,17.0,0.001504,169.0,0.0,1.0,14.893075,191.0


In [36]:
from scipy.stats import norm

WRSR = df_rank["WRSR"]
WRSR_rank_dict = dict(zip(WRSR.values,WRSR.rank().values)) # rank函数默认为平均秩数
distribution = pd.DataFrame(index = sorted(WRSR.unique()))
distribution["f"] = WRSR.value_counts().sort_index()
distribution["Σf"] = distribution["f"].cumsum()
distribution["average rank"] = [WRSR_rank_dict[i] for i in distribution.index]
distribution["average rank/n*100%"] = distribution["average rank"]/m
distribution.iloc[-1,-1] = 1 - 1/(4*n)
distribution['Probit'] = 5 - norm.isf(distribution.iloc[:, -1])
distribution

,f,Σf,average rank,average rank/n*100%,Probit
0.255048,1,1,1.0,0.002786,2.227984
0.474927,1,2,2.0,0.005571,2.461788
0.496290,1,3,3.0,0.008357,2.607041
0.559761,1,4,4.0,0.011142,2.714510
0.659355,1,5,5.0,0.013928,2.800680
...,...,...,...,...,...
31.236089,1,355,355.0,0.988858,7.285490
31.265346,1,356,356.0,0.991643,7.392959
31.341887,1,357,357.0,0.994429,7.538212
31.517749,1,358,358.0,0.997214,7.772016


In [37]:
import statsmodels.api as sm
r0 = np.polyfit(distribution['Probit'], distribution.index, deg=1) # 指定为1阶线性回归
pd.DataFrame(r0,index=["coefficient","bias"],columns=["value"])

,value
coefficient,8.966520
bias,-28.965346


In [38]:
if r0[1] > 0:
	print(f"\n回归直线方程为:y = {r0[0]} Probit + {r0[1]}")
else:
	print(f"\n回归直线方程为:y = {r0[0]} Probit - {abs(r0[1])}")


回归直线方程为:y = 8.966520195964275 Probit - 28.9653462001007


In [39]:
sm.OLS(distribution.index, sm.add_constant(distribution['Probit'])).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.962
Model:                            OLS   Adj. R-squared:                  0.962
Method:                 Least Squares   F-statistic:                     8955.
Date:                Fri, 17 Feb 2023   Prob (F-statistic):          6.43e-255
Time:                        21:06:21   Log-Likelihood:                -714.24
No. Observations:                 359   AIC:                             1432.
Df Residuals:                     357   BIC:                             1440.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -28.9653      0.483    -59.927      0.000     -29.916     -28.015
Probit         8.9665      0.095     94.632      0.000       8.780       9.153
==============================================================================
Omnibus:                       46.596   Durbin-Watson:                   0.108
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              360.558
Skew:                           0.008   Prob(JB):                     5.08e-79
Kurtosis:                       7.910   Cond. No.                         27.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
df_rank['Probit'] = df_rank['WRSR'].apply(lambda item: distribution.at[item, 'Probit'])
df_rank['WRSR Regression'] = np.polyval(r0, df_rank['Probit'])
threshold = np.polyval(r0, [2, 4, 6, 8])
grade = pd.DataFrame(pd.cut(df_rank['WRSR Regression'], threshold, labels=range(len(threshold) - 1, 0, -1)))
grade.columns = ["grade"]
grade

,grade
0,2
1,2
2,2
3,2
4,1
...,...
354,2
355,3
356,2
357,2


In [42]:
df["grade"] = grade
df.to_excel("难度分档结果.xlsx")